## Challenge: MovieLens Database Analysis

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("DataFrame").getOrCreate()

df = spark.read.option("sep", "\t").option("inferSchema", "true").option("header", "false").csv("/data/u.data")

# Ajout noms des colonnes
df = df.withColumnRenamed("_c0", "user_id").withColumnRenamed("_c1", "movieID").withColumnRenamed("_c2", "rating").withColumnRenamed("_c3", "timestamp")

# Affichage des 5 premières lignes
df.show(5)

+-------+-------+------+---------+
|user_id|movieID|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



### Ratings Count: Calculate the number of ratings received by each movie using data from u.data. // Sorting: Order the movies from the most to the least number of ratings.

In [7]:
ratings_count_df = df.groupBy("movieID") \
    .agg(F.count("rating").alias("count")) \
    .sort(F.desc("count"))

ratings_count_df.show()

+-------+-----+
|movieID|count|
+-------+-----+
|     50|  583|
|    258|  509|
|    100|  508|
|    181|  507|
|    294|  485|
|    286|  481|
|    288|  478|
|      1|  452|
|    300|  431|
|    121|  429|
|    174|  420|
|    127|  413|
|     56|  394|
|      7|  392|
|     98|  390|
|    237|  384|
|    117|  378|
|    172|  367|
|    222|  365|
|    204|  350|
+-------+-----+
only showing top 20 rows



### Name Enrichment: Utilize a broadcast variable to import movie titles from u.item and append them to your dataset.

In [8]:
# Charger le fichier u.item
movies_df = spark.read.option("sep", "|").option("inferSchema", "true").option("header", "false").csv("/data/u.item")
movies_df = movies_df.withColumnRenamed("_c0", "movieID") \
                     .withColumnRenamed("_c1", "movieTitle")

movies_df.show()

+-------+--------------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|movieID|          movieTitle|        _c2| _c3|                 _c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|
+-------+--------------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|      1|    Toy Story (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  0|  0|  1|  1|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
|      2|    GoldenEye (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  1|  1|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|      3|   Four Rooms (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  0|  0|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|      4|   Get Shorty (1995)|01-Jan-1995|NULL

In [11]:
# Créer un dictionnaire contenant les titres des films avec leurs IDs comme clés
movie_titles = {row["movieID"]: row["movieTitle"] for row in movies_df.select("movieID", "movieTitle").collect()}
#print(movie_titles)

In [12]:
# Créer une Broadcast Variable pour les titres de films
broadcastVar = spark.sparkContext.broadcast(movie_titles)

In [13]:
# Créer une UDF pour récupérer le titre du film à partir de l'ID
def get_movie_title(movieID):
    return broadcastVar.value.get(movieID, "Unknown")

In [14]:
get_movie_title_UDF = F.udf(get_movie_title)

In [18]:
# Ajouter le titre du film au DataFrame des comptes de notes
df_with_titles = ratings_count_df.withColumn("movieTitle", get_movie_title_UDF(ratings_count_df["movieID"]))

In [19]:
df_with_titles.show()

+-------+-----+--------------------+
|movieID|count|          movieTitle|
+-------+-----+--------------------+
|     50|  583|    Star Wars (1977)|
|    258|  509|      Contact (1997)|
|    100|  508|        Fargo (1996)|
|    181|  507|Return of the Jed...|
|    294|  485|    Liar Liar (1997)|
|    286|  481|English Patient, ...|
|    288|  478|       Scream (1996)|
|      1|  452|    Toy Story (1995)|
|    300|  431|Air Force One (1997)|
|    121|  429|Independence Day ...|
|    174|  420|Raiders of the Lo...|
|    127|  413|Godfather, The (1...|
|     56|  394| Pulp Fiction (1994)|
|      7|  392|Twelve Monkeys (1...|
|     98|  390|Silence of the La...|
|    237|  384|Jerry Maguire (1996)|
|    117|  378|    Rock, The (1996)|
|    172|  367|Empire Strikes Ba...|
|    222|  365|Star Trek: First ...|
|    204|  350|Back to the Futur...|
+-------+-----+--------------------+
only showing top 20 rows



In [20]:
# Arrêt de la SparkSession
spark.stop()